<a href="https://colab.research.google.com/github/shreyadwivedi14/shreyadwivedi14/blob/main/CSCI_544_HW3_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
# Shreya Dwivedi
# 5148430435
# HW3

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install contractions
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.6 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import contractions
import gensim
import gensim.downloader as api
from gensim.test.utils import datapath
from gensim import utils
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [7]:
import pandas as pd

file_path = "/content/drive/My Drive/Colab Notebooks/CSCI_544/HW1/amazon_reviews_us_Office_Products_v1_00.tsv"

# Skip lines with parsing errors
df = pd.read_csv(file_path, delimiter='\t', error_bad_lines=False)

Skipping line 20773: expected 15 fields, saw 22
Skipping line 39834: expected 15 fields, saw 22
Skipping line 52957: expected 15 fields, saw 22
Skipping line 54540: expected 15 fields, saw 22

Skipping line 80276: expected 15 fields, saw 22
Skipping line 96168: expected 15 fields, saw 22
Skipping line 96866: expected 15 fields, saw 22
Skipping line 98175: expected 15 fields, saw 22
Skipping line 112539: expected 15 fields, saw 22
Skipping line 119377: expected 15 fields, saw 22
Skipping line 120065: expected 15 fields, saw 22
Skipping line 124703: expected 15 fields, saw 22

Skipping line 134024: expected 15 fields, saw 22
Skipping line 153938: expected 15 fields, saw 22
Skipping line 156225: expected 15 fields, saw 22
Skipping line 168603: expected 15 fields, saw 22
Skipping line 187002: expected 15 fields, saw 22

Skipping line 200397: expected 15 fields, saw 22
Skipping line 203809: expected 15 fields, saw 22
Skipping line 207680: expected 15 fields, saw 22
Skipping line 223421: exp

In [8]:
 df = df[['star_rating','review_headline','review_body']]

In [9]:
df['star_rating']=pd.to_numeric(df['star_rating'], errors='coerce')

In [10]:
filtered_df = df[pd.notna(df['star_rating'])]

#### We form two classes and select 50000 reviews randomly from each class.

In [11]:
# Creating class
filtered_df['class'] = (filtered_df['star_rating'].astype(int)).apply(lambda x: 'class1' if x in [1, 2, 3] else 'class2')

In [12]:
filtered_df.head()

,star_rating,review_headline,review_body,class
0,5.0,Five Stars,Great product.,class2
1,5.0,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,class2
2,5.0,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",class2
3,1.0,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,class1
4,4.0,Four Stars,Gorgeous colors and easy to use,class2


In [13]:
balanced_df = pd.DataFrame(columns = filtered_df.columns)

In [14]:
class_subset1 = filtered_df[filtered_df['class'] == 'class1'].sample(50000, random_state=42)
balanced_df = balanced_df.append(class_subset1)
class_subset2 = filtered_df[filtered_df['class'] == 'class2'].sample(50000, random_state=42)
balanced_df = balanced_df.append(class_subset2)

In [15]:
balanced_df.head()

,star_rating,review_headline,review_body,class
206428,1.0,False advertisement! Doesn't work with lifepro...,it says right in the specifications that it wo...,class1
104177,1.0,horrible product,These things were horrible. Used 6 to hang a ...,class1
1610066,2.0,Doesn't work for my Epson printer,"Not sure why, but works for one Epson printer ...",class1
2489071,1.0,Not received,I ordered two letter organizers and received t...,class1
1505472,3.0,5 gears are good but 7 would have been better,"This kit comes with 5 gears, 4 of which go int...",class1


In [16]:
review_text = balanced_df['review_body'].astype(str).apply(gensim.utils.simple_preprocess)

In [17]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(balanced_df['review_body'], balanced_df['class'], test_size=0.2, random_state=42)

In [18]:
X_train = X_train.astype(str).apply(gensim.utils.simple_preprocess)

In [19]:
X_test = X_test.astype(str).apply(gensim.utils.simple_preprocess)

### 2. Word Embedding

#### 2 (a)

In [20]:
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [21]:
pairs = [
    ('King', 'Man'),   # a minivan is a kind of car
    ('Queen', 'Woman'),
    ('excellent', 'outstanding')
]

In [22]:
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'King'	'Man'	0.18
'Queen'	'Woman'	0.17
'excellent'	'outstanding'	0.56


In [23]:
vec_king = wv['king']
vec_queen = wv['queen']
vec_man = wv['man']
vec_woman = wv['woman']

In [24]:
# Calculate the vector for 'queen' using vector arithmetic
vec_queen_calc = vec_king - vec_man + vec_woman

# Reshape the vectors for similarity calculation
vec_queen_calc = vec_queen_calc.reshape(1, -1)
vec_queen = vec_queen.reshape(1, -1)

# Calculate cosine similarity
similarity = cosine_similarity(vec_queen, vec_queen_calc)
print('Cosine similarity between calculated "queen" vector and original "queen" vector:', similarity[0][0])

Cosine similarity between calculated "queen" vector and original "queen" vector: 0.7300518


#### 2(b)

In [25]:
model = gensim.models.Word2Vec(
    vector_size = 300,
    window = 13,
    min_count=9,
    workers=4,
)

In [26]:
# Building Vocab
model.build_vocab(review_text, progress_per=1000)

In [27]:
# Train the Word2Vec Model
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(20194068, 27325360)

In [28]:
# Checking similarity between words
model.wv.similarity(w1="excellent", w2="outstanding")

0.80513096

In [29]:
vec_king = model.wv['king']
vec_queen = model.wv['queen']
vec_man = model.wv['man']
vec_woman = model.wv['woman']

In [30]:
# Calculate the vector for 'queen' using vector arithmetic
vec_queen_calc = vec_king - vec_man + vec_woman

# Reshape the vectors for similarity calculation
vec_queen_calc = vec_queen_calc.reshape(1, -1)
vec_queen = vec_queen.reshape(1, -1)

# Calculate cosine similarity
similarity = cosine_similarity(vec_queen, vec_queen_calc)
print('Cosine similarity between calculated "queen" vector and original "queen" vector:', similarity[0][0])

Cosine similarity between calculated "queen" vector and original "queen" vector: 0.17913644


### 3. Simple models

In [31]:
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC

In [32]:
perceptron = Perceptron()
svm = SVC()

In [33]:
# Generate average Word2Vec vectors for each review
def generate_avg_word2vec_vectors(review, model):
    vectors = [model[word] for word in review if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(300)

In [34]:
# Train and predict using Word2Vec features
X_train_word2vec = np.array([generate_avg_word2vec_vectors(review, wv) for review in X_train])
X_test_word2vec = np.array([generate_avg_word2vec_vectors(review, wv) for review in X_test])


perceptron.fit(X_train_word2vec, y_train)
svm.fit(X_train_word2vec, y_train)
y_pred_perceptron_word2vec = perceptron.predict(X_test_word2vec)
y_pred_svm_word2vec = svm.predict(X_test_word2vec)

In [35]:
# Calculate Precision, Recall, and F1-score
precision_svm = precision_score(y_test, y_pred_svm_word2vec, pos_label ='class1') # Replace y_true with your true labels
recall_svm = recall_score(y_test, y_pred_svm_word2vec, pos_label ='class1') # Replace y_true with your true labels
f1_svm = f1_score(y_test, y_pred_svm_word2vec, pos_label ='class1')  # Replace y_true with your true labels

# Print the results
print(f'Precision: {precision_svm}')
print(f'Recall: {recall_svm}')
print(f'F1-score: {f1_svm}')

Precision: 0.8269012485811578
Recall: 0.8711509715994021
F1-score: 0.8484495559761246


In [36]:
# Calculate Precision, Recall, and F1-score
precision_per = precision_score(y_test, y_pred_perceptron_word2vec, pos_label ='class1') # Replace y_true with your true labels
recall_per = recall_score(y_test, y_pred_perceptron_word2vec, pos_label ='class1') # Replace y_true with your true labels
f1_per = f1_score(y_test, y_pred_perceptron_word2vec, pos_label ='class1')  # Replace y_true with your true labels

# Print the results
print(f'Precision: {precision_per}')
print(f'Recall: {recall_per}')
print(f'F1-score: {f1_per}')

Precision: 0.9229900632339657
Recall: 0.40727453911310413
F1-score: 0.5651662863859503


### 4. Feedforward Neural Networks

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [38]:
# Convert labels to numerical values
label_mapping = {'class1': 0, 'class2': 1}
y_train_numeric = y_train.map(label_mapping)
y_test_numeric = y_test.map(label_mapping)

In [39]:
print("Unique values in y_train:", np.unique(y_train.values))
print("Unique values in y_test:", np.unique(y_test.values))


Unique values in y_train: ['class1' 'class2']
Unique values in y_test: ['class1' 'class2']


In [40]:
# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_word2vec, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_word2vec, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_numeric.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_numeric.values, dtype=torch.long)

In [41]:
# Define vec_size
vec_size = 300  # Assuming Word2Vec vectors are of size 300

# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(vec_size, 50)
        self.fc2 = nn.Linear(50, 5)
        self.fc3 = nn.Linear(5, 2)  # Output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [42]:
# Initialize the model, loss function, and optimizer
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [43]:
# Define the DataLoader for training and testing sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [44]:
# Train the model
num_epochs = 10  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [45]:
# Evaluate the model on the testing set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Accuracy on the testing set:', accuracy)

Accuracy on the testing set: 0.84175


#### 4(b)

In [46]:
# Concatenate the first 10 Word2Vec vectors for each review
def concatenate_word2vec_vectors(review_text, num_vectors=10):
    concatenated_vectors = []
    for review in review_text:
        review_vectors = [wv[word] for word in review[:num_vectors] if word in wv]
        if len(review_vectors) < num_vectors:
            # If a review has less than num_vectors words in Word2Vec, pad with zeros
            review_vectors.extend([np.zeros(300)] * (num_vectors - len(review_vectors)))
        concatenated_vectors.append(np.concatenate(review_vectors))
    return np.array(concatenated_vectors)

In [47]:
# Concatenate the first 10 Word2Vec vectors for training and testing data
X_train_concat = concatenate_word2vec_vectors(X_train_word2vec)
X_test_concat = concatenate_word2vec_vectors(X_test_word2vec)

# Convert to PyTorch tensors
X_train_tensor_concat = torch.tensor(X_train_concat, dtype=torch.float32)
X_test_tensor_concat = torch.tensor(X_test_concat, dtype=torch.float32)

In [48]:
class MLPConcat(nn.Module):
    def __init__(self):
        super(MLPConcat, self).__init__()
        self.fc1 = nn.Linear(3000, 100)  # Increased units to 100
        self.fc2 = nn.Linear(100, 20)  # Increased units to 20
        self.fc3 = nn.Linear(20, 2)  # Output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [49]:
# Initialize the model, loss function, and optimizer for updated model
model_concat_updated = MLPConcat()
criterion_concat = nn.CrossEntropyLoss()
optimizer_concat = optim.Adam(model_concat_updated.parameters(), lr=0.001)

In [50]:
# Define the DataLoader for training and testing sets
train_dataset = TensorDataset(X_train_tensor_concat, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_tensor_concat, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [51]:
num_epochs = 10  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
    model_concat_updated.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_concat_updated(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [52]:
# Evaluate the model on the testing set
model_concat_updated.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_concat_updated(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Accuracy on the testing set:', accuracy)

Accuracy on the testing set: 0.50175


### 5. Recurrent Neural Networks

In [92]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize the initial hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward pass
        out, _ = self.rnn(x, h0)

        # Index hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out

In [85]:
def preprocess_word2vec_features(data, max_length):
    """
    Preprocesses Word2Vec features by truncating longer reviews and padding shorter reviews with zero vectors.

    Parameters:
    data (numpy.ndarray): Word2Vec features for reviews.
    max_length (int): Maximum review length.

    Returns:
    numpy.ndarray: Preprocessed Word2Vec features.
    """
    processed_data = []
    for review_vectors in data:
        if len(review_vectors) > max_length:
            processed_data.append(review_vectors[:max_length])
        elif len(review_vectors) < max_length:
            # Pad with zero vectors
            padding_length = max_length - len(review_vectors)
            padding_vectors = np.zeros((padding_length, review_vectors.shape[1]))
            processed_data.append(np.vstack((review_vectors, padding_vectors)))
        else:
            processed_data.append(review_vectors)
    return np.array(processed_data)

In [93]:
# Preprocess the Word2Vec features
max_length = 10  # Maximum review length
X_train_rnn = preprocess_word2vec_features(X_train_word2vec, max_length)
X_test_rnn = preprocess_word2vec_features(X_test_word2vec, max_length)

In [94]:
# Convert data to PyTorch tensors
X_train_tensor_rnn = torch.tensor(X_train_rnn, dtype=torch.float32)
X_test_tensor_rnn = torch.tensor(X_test_rnn, dtype=torch.float32)
y_train_tensor_rnn = torch.tensor(y_train_numeric.values, dtype=torch.long)
y_test_tensor_rnn = torch.tensor(y_test_numeric.values, dtype=torch.long)

In [95]:
# Initialize the RNN model with num_layers
rnn_model = SimpleRNN(input_size=300, hidden_size=10, num_layers=1, output_size=2)


In [96]:
# Define loss and optimizer
criterion_rnn = nn.CrossEntropyLoss()
optimizer_rnn = torch.optim.Adam(rnn_model.parameters(), lr=0.001)

In [97]:
# Define the DataLoader for training and testing sets
train_dataset_rnn = TensorDataset(X_train_tensor_rnn, y_train_tensor_rnn)
train_loader_rnn = DataLoader(train_dataset_rnn, batch_size=32, shuffle=True)
test_dataset_rnn = TensorDataset(X_test_tensor_rnn, y_test_tensor_rnn)
test_loader_rnn = DataLoader(test_dataset_rnn, batch_size=32, shuffle=False)

In [100]:
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming vec_size is the correct input size for the RNN model (e.g., 300)
vec_size = 300

# Train the RNN model
num_epochs = 10

for epoch in range(num_epochs):
    rnn_model.train()

    for inputs, labels in train_loader_rnn:
        # Check if the inputs have the correct length
        if inputs.shape[1] != vec_size:
            # If not, pad or truncate to the correct length
            pad_width = max(0, vec_size - inputs.shape[1])
            inputs = F.pad(inputs, (0, pad_width))

        # Reshape inputs to match the expected dimensions (batch_size, sequence_length, input_size)
        inputs = inputs.unsqueeze(1)  # Add a dummy sequence length dimension
        inputs = inputs.repeat(1, vec_size, 1)  # Repeat to match sequence length
        inputs = inputs.permute(0, 2, 1)  # Permute dimensions

        # Zero the gradients
        optimizer_rnn.zero_grad()

        # Forward pass
        outputs = rnn_model(inputs)

        # Compute the loss
        loss = criterion_rnn(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer_rnn.step()


In [102]:
# Evaluate the RNN model
rnn_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader_rnn:
        # Check if the inputs have the correct length
        if inputs.shape[1] != vec_size:
            pad_width = max(0, vec_size - inputs.shape[1])
            inputs = F.pad(inputs, (0, pad_width))

        # Reshape inputs to match the expected dimensions (batch_size, sequence_length, input_size)
        inputs = inputs.unsqueeze(1)
        inputs = inputs.repeat(1, vec_size, 1)
        inputs = inputs.permute(0, 2, 1)

        outputs = rnn_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_rnn = correct / total
print('Accuracy on the testing set (RNN):', accuracy_rnn)


Accuracy on the testing set (RNN): 0.50175
